# MDE

## *Ноутбук подготовлен на основе публикации "Когда останавливать A/B-тест? Часть 1: MDE"*

In [1]:
import numpy as np
import pandas as pd
import random
from scipy.stats import norm
from statsmodels.stats.power import tt_ind_solve_power 

Минимальный ожидаемый эффект — это наименьший истинный эффект полученный от изменений, который с уверенностью сможет обнаружить статистический критерий.
Если прокраса в метрике нет, то это не означает отсутствие эффекта. Эффект может и есть, но он точно не больше, чем MDE для α, β и дисперсии.

Ошибка второго рода (beta, False Negative) — принятие неверной нулевой гипотезы. Вероятность ошибочно сохранить неверную нулевую гипотезу обозначают греческой буквой β (например, 0.2)

Ошибка первого рода (alpha, False positive) — отклонение верной нулевой гипотезы. Риск совершить такую ошибку равен выбранному 1-уровень статистической значимости и обозначают греческой буквой α (например, α=0.05)

Мощность true positive) — вероятность, что тест правильно засечёт эффект там, где он и правда есть. (1-β)

Статистическая значимость (True Negative) — вероятность, что результат эксперимента получился случайно и мы это знаем. Другими словами, это риск, что тест покажет взаимосвязь, которой на самом деле нет (1-α)

In [2]:
mean = 1.8 # рассчитанное среднее за N недель
print(f'mean - {mean}')
std = 1 # рассчитанное стандартное отклонение за тот же период
print(f'std - {std}')
n = 500
print(f'n - {n}')
alpha = 0.05 # уровень значимости не менее 95% (чем больше – тем лучше)
print(f'alpha - {alpha}')
power = 0.8 # уровень мощности не менее 80%
print(f'power- {power}')
lift = 0.1 # от 0.01 до 1.00 (где 1.00 – 100% прироста у метрики)
print(f'lift- {lift}')

mean - 1.8
std - 1
n - 500
alpha - 0.05
power- 0.8
lift- 0.1


### Effect Size

In [3]:
effect_size = mean / std * lift
print(f'effect_size - {effect_size}')

effect_size - 0.18000000000000002


### Estimate Effect Size. MDE. Вариант 1

Допустим, мы запустили A/B. Было накоплено по ~500 наблюдений в каждой ветке эксперимента. Мы хотим проверить метрику с непрерывным распределением и узнать какой “точности” мы добились за это время.

In [4]:
def estimate_effect_size_v1(std, n, alpha=alpha, power=power):
    """
    Расчет MDE для баланса 50/50
    :param sd: ско одной группы
    :param n: размер выборки в одной группе
    :return: MDE
    """
    S = np.sqrt((std**2 / n) + (std**2 / n))
    M = norm.ppf(q=1-alpha/2) + norm.ppf(q=power)
    return M * S

In [5]:
mde_v1 = estimate_effect_size_v1(std, n)
print(f'MDE - {mde_v1}')

MDE - 0.17718780696593192


Мы получили в качестве результата 0.17. Это означает, что эффекты ниже этого значения будут обладать меньшей мощностью. Соответственно, вероятность того, что эффект действительно есть будет меньше. Эффекты выше этого значения, наоборот, будут мощнее.

### Estimate Effect Size. MDE. Вариант 2

In [6]:
def estimate_effect_size_v2(std, n, alpha=alpha, beta=(1-power)):
    t_alpha = norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = norm.ppf(1 - beta, loc=0, scale=1)
    disp_sum_sqrt = (2 * (std ** 2)) ** 0.5
    mde = (t_alpha + t_beta) * disp_sum_sqrt / np.sqrt(n)
    return mde

In [7]:
mde_v2 = estimate_effect_size_v2(std, n)
print(f'MDE - {mde_v2}')

MDE - 0.17718780696593192


### Estimate Effect Size. MDE. Вариант 3

In [8]:
mde_v3 = tt_ind_solve_power(nobs1=n, alpha=alpha, power=power, ratio=1)
print(f'MDE - {mde_v3}')

MDE - 0.17735842307242328


### Estimate Sample Size. Вариант 1

Теперь сделаем обратную операцию и рассчитаем количество наблюдений, зная Effect Size. При тех же значениях alpha и power, получим тот же самый размер выборки, что был ранее в качестве входного параметра:

In [9]:
def estimate_sample_size(effect_size, std, alpha=alpha, power=power):
    """
    Расчет N для баланса 50/50
    :param sd: ско одной группы
    :param effect_size: ожидания по изменения в метрику
    :return: N
    """
    M = (norm.ppf(q=1-alpha/2) + norm.ppf(q=power))**2
    return 2 * M * std**2 / effect_size**2 

In [10]:
# Результат – наблюдения на одну выборку!
nobs_v1 = estimate_sample_size(mde_v1, std)
print(f'nobs - {nobs_v1}')

nobs - 500.0


Ранее было сказано, что эффекты ниже 0.17 будут обладать меньшей мощностью при 500 наблюдениях. Следовательно, больше наблюдений → больше мощность для того же эффекта и выше.

### Estimate Sample Size. Вариант 2

In [11]:
nobs_v2 = tt_ind_solve_power(effect_size=mde_v1, alpha=alpha, power=power, nobs1=None, ratio=1)
print(f'nobs - {nobs_v2}')

nobs - 500.96095684136526


### Estimate Sample Size. Вариант 3

In [12]:
nobs_v3 = tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power, nobs1=None, ratio=1)
print(f'nobs - {nobs_v3}')

nobs - 485.4598017596581
